In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import random
from PIL import Image
import PIL.ImageDraw as ImageDraw
from matplotlib.patches import Polygon, Rectangle

In [2]:
root = os.getcwd()

In [15]:
annots_data = os.listdir(root + '/data')
imgs_data = os.listdir(root + '/images')

annots_data.sort()
imgs_data.sort()


In [18]:
def get_labels(annot):
    """returns the labels needed to draw the polygons, bounding box
    and denormalize the points
    """
    labels = []
    points = []
    width = 0
    height = 0
    for an in annot:
        if 'polygonlabels' in an['value']:
            labels.append(an['value']['polygonlabels'][0])
            points.append(an['value']['points'])
            width = int(an['original_width'])
            height = int(an['original_height'])
    return labels, points, width, height

In [21]:
def create_xy(coords, width, height):
    """returns a tuple with the normalized xy points in the
    needed format for polygons
    """
    coords.append(coords[0])
    x, y = zip(*coords)
    x = np.asarray(x)
    y = np.asarray(y)
    norm_x = ( x / 100 ) * width
    norm_y = (y / 100 ) * height
    return (norm_x, norm_y)

In [20]:
def bounding_box_naive(points):
    """returns a list containing the bottom left and the top right 
    points in the sequence
    Here, we use min and max four times over the collection of points
    """
    bot_left_x = min(point[0] for point in points)
    bot_left_y = min(point[1] for point in points)
    top_right_x = max(point[0] for point in points)
    top_right_y = max(point[1] for point in points)

    return [(bot_left_x, bot_left_y), (top_right_x, top_right_y)]

In [24]:
def lighten_color(color, amount=0.3):
    """returns a lighten color for the polygons
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

In [15]:
def draw_annots(image_path, labels, points, width, height, alpha=0.5, t=1):
    """display the annotated image with polygons, bounding_box and label
    The alpha value controls the transparecy
    """
    colors = ['r','g','b','c','m','y']

    fig, ax = plt.subplots()
    image = Image.open(image_path)
    ax.imshow(image)
    
    for label, p in zip(labels, points):
        x,y = create_xy(p, width, height) #'#702219'
        color = "#%06x" % random.randint(0, 0xFFFFFF)
        border_color = random.choice(colors)
        fill_color = lighten_color(border_color)
        ax.add_patch(Polygon(np.vstack((x,y)).T, fill=True, color=fill_color, ec=border_color,
                        alpha=alpha))
        if t==2:
            bb = bounding_box_naive(np.vstack((x,y)).T)
            ax.add_patch(Rectangle(bb[0], bb[1][0]-bb[0][0], bb[1][1]-bb[0][1], linewidth=1, edgecolor=border_color, facecolor='none'))
            ax.annotate(label, bb[0], color='black', weight='bold', fontsize=9)
            
    return ax

In [18]:
def draw_image(image_path,  meta_data_json_path, alpha):
    
    file = open(meta_data_json_path)
    annot = json.load(file)
    
    labels, points, width, height = get_labels(annot)
    type_1 = draw_annots(image_path, labels, points, width, height, alpha, t=1)
    type_2 = draw_annots(image_path, labels, points, width, height, alpha, t=2)
    
    type_1 = fig2img(type_1.figure)
    type_2 = fig2img(type_2.figure)
    
    return [type_1, type_2]
    

In [19]:
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf)
    return img
